In [1]:
from chi import server, context, lease
import chi, os, time, datetime

context.version = "1.0" 
context.choose_project()
context.choose_site(default="CHI@TACC")

In [2]:
l = lease.get_lease(f"Project24001") 
l.show()

HTML(value='\n        <h2>Lease Details</h2>\n        <table>\n            <tr><th>Name</th><td>Project24001</…

Lease Details:
Name: Project24001
ID: 1f85ceb3-0227-4f6d-af26-667358395a1c
Status: ACTIVE
Start Date: 2025-05-10 01:17:00
End Date: 2025-05-10 02:20:00
User ID: e2577330508e3973cfe043571304aac84303c56c5f6e3388146f5a29a70065c7
Project ID: d3c6e101843a4ba79e665ebf59b521a2

Node Reservations:
ID: 68fe0fa6-e070-4ea9-a51a-d07b47ff5c6c, Status: active, Min: 1, Max: 1

Floating IP Reservations:

Network Reservations:

Events:


In [ ]:
username = os.getenv('USER') # all exp resources will have this prefix
s = server.Server(
    f"node-sb9880-01", 
    reservation_id=l.node_reservations[0]["id"],
    image_name="CC-Ubuntu24.04-CUDA"
)
s.submit(idempotent=True)

Waiting for server node-sb9880-01's status to become ACTIVE. This typically takes 10 minutes, but can take up to 20 minutes.


In [ ]:
s.associate_floating_ip()

In [ ]:
s.refresh()
s.check_connectivity()

s.refresh()
s.show(type="widget")

By default, all connections to VM resources are blocked, as a security measure. We need to attach one or more “security groups” to our VM resource, to permit access over the Internet to specified ports.

The following security groups will be created (if they do not already exist in our project) and then added to our server:


In [ ]:
security_groups = [
  {'name': "allow-ssh", 'port': 22, 'description': "Enable SSH traffic on TCP port 22"},
  {'name': "allow-8888", 'port': 8888, 'description': "Enable TCP port 8888 (used by Jupyter)"},
  {'name': "allow-8000", 'port': 8000, 'description': "Enable TCP port 8000 (used by MLFlow)"},
  {'name': "allow-9000", 'port': 9000, 'description': "Enable TCP port 9000 (used by MinIO API)"},
  {'name': "allow-9001", 'port': 9001, 'description': "Enable TCP port 9001 (used by MinIO Web UI)"}
]

In [ ]:
# configure openstacksdk for actions unsupported by python-chi
os_conn = chi.clients.connection()
nova_server = chi.nova().servers.get(s.id)

for sg in security_groups:

  if not os_conn.get_security_group(sg['name']):
      os_conn.create_security_group(sg['name'], sg['description'])
      os_conn.create_security_group_rule(sg['name'], port_range_min=sg['port'], port_range_max=sg['port'], protocol='tcp', remote_ip_prefix='0.0.0.0/0')

  nova_server.add_security_group(sg['name'])

print(f"updated security groups: {[group.name for group in nova_server.list_security_group()]}")

In [ ]:
# load git repo
#s.execute("git clone --recurse-submodules https://github.com/AryanAjmera18/EYE")


Set up Docker

In [ ]:
s.execute("curl -sSL https://get.docker.com/ | sudo sh")
s.execute("sudo groupadd -f docker; sudo usermod -aG docker $USER")

## For nvidia GPU

In [ ]:
s.execute("curl -fsSL https://nvidia.github.io/libnvidia-container/gpgkey | sudo gpg --dearmor -o /usr/share/keyrings/nvidia-container-toolkit-keyring.gpg \
  && curl -s -L https://nvidia.github.io/libnvidia-container/stable/deb/nvidia-container-toolkit.list | \
    sed 's#deb https://#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://#g' | \
    sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list")
s.execute("sudo apt update")
s.execute("sudo apt-get install -y nvidia-container-toolkit")
s.execute("sudo nvidia-ctk runtime configure --runtime=docker")
# for https://github.com/NVIDIA/nvidia-container-toolkit/issues/48
s.execute("sudo jq 'if has(\"exec-opts\") then . else . + {\"exec-opts\": [\"native.cgroupdriver=cgroupfs\"]} end' /etc/docker/daemon.json | sudo tee /etc/docker/daemon.json.tmp > /dev/null && sudo mv /etc/docker/daemon.json.tmp /etc/docker/daemon.json")
s.execute("sudo systemctl restart docker")

In [ ]:
s.execute("sudo apt update")
s.execute("sudo apt -y install nvtop")

## For AMD GPU
Before we can use the AMD GPUs, we need to set up the driver using the `amdgpu-install` utility.

Let’s follow [AMD’s instructions for setting up `amdgpu-install`](https://rocm.docs.amd.com/projects/install-on-linux/en/latest/install/install-methods/amdgpu-installer/amdgpu-installer-ubuntu.html#installation):

In [ ]:
s.execute("sudo apt update; wget https://repo.radeon.com/amdgpu-install/6.3.3/ubuntu/noble/amdgpu-install_6.3.60303-1_all.deb")
s.execute("sudo apt -y install ./amdgpu-install_6.3.60303-1_all.deb; sudo apt update")

In [ ]:
s.execute("amdgpu-install -y --usecase=dkms")

In [ ]:
s.execute("sudo apt -y install rocm-smi")

In [ ]:
s.execute("sudo usermod -aG video,render $USER")

In [ ]:
s.execute("sudo reboot")
time.sleep(30)

In [ ]:
s.refresh()
s.check_connectivity()

In [ ]:
s.execute("rocm-smi")

In [ ]:
s.execute("sudo apt -y install cmake libncurses-dev libsystemd-dev libudev-dev libdrm-dev libgtest-dev")
s.execute("git clone https://github.com/Syllo/nvtop")
s.execute("mkdir -p nvtop/build && cd nvtop/build && cmake .. -DAMDGPU_SUPPORT=ON && sudo make install")

## build container image for mlflow section

In [ ]:
s.execute("git clone --recurse-submodules https://github.com/sb9880/Data_eye")
s.execute("docker build -t jupyter-mlflow -f Data_eye/Dockerfile.jupyter-torch-mlflow-cuda .")

1st lab mlflow done - 
## Now we can SSH
ssh -i ~/.ssh/id_rsa_chameleon cc@129.114.108.94




1. Get container-Object id 
2. Install rclone
##### run on node-persist
curl https://rclone.org/install.sh | sudo bash
    
    # run on node-persist
##### this line makes sure user_allow_other is un-commented in /etc/fuse.conf
sudo sed -i '/^#user_allow_other/s/^#//' /etc/fuse.conf

Next, create a configuration file for rclone with the ID and secret from the application credential you just generated:

# run on node-persist
mkdir -p ~/.config/rclone
nano  ~/.config/rclone/rclone.conf
Paste the following into the config file, but substitute your own application credential ID and secret.

You will also need to substitute your own user ID. You can find it using “Identity” > “Users” in the Horizon GUI; it is an alphanumeric string (not the human-readable user name).

[chi_tacc]
type = swift
user_id = YOUR_USER_ID
application_credential_id = APP_CRED_ID
application_credential_secret = APP_CRED_SECRET
auth = https://chi.tacc.chameleoncloud.org:5000/v3
region = CHI@TACC
Use Ctrl+O and Enter to save the file, and Ctrl+X to exit nano.

To test it, run

# run on node-persist
rclone lsd chi_tacc:
and verify that you see your container listed. This confirms that rclone can authenticate to the object store.

---------> object-persist-project24
### ETL loads data to object
#### Mount
The next step is to create a mount point for the data in the local filesystem:

``` bash
# run on node-persist
sudo mkdir -p /mnt/object
sudo chown -R cc /mnt/object
sudo chgrp -R cc /mnt/object
```

Now finally, we can use `rclone mount` to mount the object store at the mount point (substituting your own **netID** in the command below).

``` bash
### run on node-persist
rclone mount chi_tacc:object-persist-project24 /mnt/object --read-only --allow-other --daemon
```

Here,

-   `chi_tacc` tells `rclone` which section of its configuration file to use for authentication information
-   `object-persist-netID` tells it what object store container to mount
-   `/mnt/object` says where to mount it

Since we only intend to read the data, we can mount it in read-only mode and it will be slightly faster; and we are also protected from accidental writes. We also specified `--allow-other` so that we can use the mount from Docker, and `--daemon` means the `rclone` process will be started in the background.

Run

``` bash
# run on node-persist
ls /mnt/object
```

and confirm that we can now see the Food11 data directories (`evaluation`, `training`, `validation`) there.
-------------------------
## Mount to Docker
Now, we can start a Docker container with access to that virtual “filesystem”, by passing that directory as a bind mount. 

Note that to mount a directory that is actually a FUSE filesystem inside a Docker container, we have to pass it using a slightly different `--mount` syntax, instead of the `-v` that we had used in previous examples.



In [ ]:
First, create the volume:

``` bash
# runs on node-mltrain
docker volume create EYE
```

Then, to populate it with data, run


# runs on node-mltrain
docker compose -f /Data_eye/Docker/docker-compose-data.yaml up -d

This will run a temporary container that downloads the Food-11 dataset, organizes it in the volume, and then stops. It may take a minute or two. You can verify with

``` bash
# runs on node-mltrain
docker ps
```

that it is done - when there are no running containers.

Finally, verify that the data looks as it should. Start a shell in a temporary container with this volume attached, and `ls` the contents of the volume:

``` bash
# runs on node-mltrain
docker run --rm -it -v EYE:/mnt alpine ls -l /mnt/eye_dataset/



```

it should show “evaluation”, “validation”, and “training” subfolders.

### Un-mount an object store

We’ll keep working with this object store in the next part, so you do not have to un-mount it now. But generally speaking to stop `rclone` running and un-mount the object store, you would run

    fusermount -u /mnt/object

where you specify the path of the mount point.